# Functional Data Gathering - Areas

## Import modules

In [1]:
from caveclient import CAVEclient
import nglui
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
from microns_phase3 import nda, utils

Connecting microns@db.datajoint.com:3306


## Building the dataframe of all functional cells

In [2]:
client = CAVEclient()

client.auth.save_token(token="a80f7690d93ead1b46e980628df5beae",overwrite = True)

# this is the datastack name of the public release
# passing it will auto-configure many of the services
client = CAVEclient('minnie65_public_v343')

#visualize available tables
client.materialize.get_tables()

['synapses_pni_2',
 'nucleus_neuron_svm',
 'nucleus_detection_v0',
 'functional_coreg',
 'allen_v1_column_types_slanted',
 'aibs_column_nonneuronal',
 'func_unit_em_match_release',
 'proofreading_status_public_release',
 'aibs_soma_nuc_metamodel_preds_v117']

In [3]:
df=client.materialize.query_table('functional_coreg') #dataframe of the functionally co-registered neurons
df.head() #visualize first couple neurons

Using deprecated pyarrow serialization method, please upgrade CAVEClient>=5.9.0 with pip install --upgrade caveclient


,id,valid,pt_supervoxel_id,pt_root_id,session,scan_idx,unit_id,pt_position
0,1,t,102531727972419182,864691136136642173,4,7,665,"[274864, 94064, 22046]"
1,2,t,94792540099826672,864691136208993724,4,7,740,"[218320, 104320, 19016]"
2,4,t,93946810036875732,864691135683934135,4,7,782,"[212128, 94544, 23423]"
3,5,t,84730565597057981,864691135334528233,4,7,855,"[145158, 110036, 19355]"
4,6,t,110905058841147285,864691136309730650,4,7,921,"[335744, 90128, 22805]"


This cell gives us the relevent identification values for each of the neurons in the co-registration subset so they can be used as inputs when gathering the activity data

All neurons measured in session 9 are removed due to issues when fetching the activity

In [4]:
session, scan_idx, unit_id, brain_area = (nda.AreaMembership & df).fetch('session','scan_idx','unit_id', 'brain_area')
df_areas = pd.DataFrame({'session': session, 'scan_idx': scan_idx, 'unit_id': unit_id, 'brain_area': brain_area})
df_areas = df_areas.loc[df_areas['session'] != 9]
df_areas

,session,scan_idx,unit_id,brain_area
0,4,7,648,RL
1,4,7,662,V1
2,4,7,664,V1
3,4,7,665,RL
4,4,7,671,V1
...,...,...,...,...
8190,8,5,10489,V1
8191,8,5,10542,V1
8192,8,5,10550,RL
8193,8,5,10567,RL


## Area V1

In [5]:
df_area_v1 = df_areas.loc[df_areas['brain_area'] == 'V1']
df_area_v1

,session,scan_idx,unit_id,brain_area
1,4,7,662,V1
2,4,7,664,V1
4,4,7,671,V1
5,4,7,682,V1
6,4,7,703,V1
...,...,...,...,...
8185,8,5,10458,V1
8186,8,5,10463,V1
8189,8,5,10480,V1
8190,8,5,10489,V1


In [ ]:
v1t1, v1t2, v1t3, v1t4, v1t5, v1t6, v1t7, v1t8, v1t9, v1t10 = ([] for i in range(10))
for row in range(len(df_area_v1)):
    unit_key = {'session': df_area_v1.iloc[row]['session'], 'scan_idx': df_area_v1.iloc[row]['scan_idx'], 'unit_id': df_area_v1.iloc[row]['unit_id']}
    oracle_traces, scores = utils.fetch_oracle_raster(unit_key)
    v1t1.append(oracle_traces[0].flatten())
    v1t2.append(oracle_traces[1].flatten())
    v1t3.append(oracle_traces[2].flatten())
    v1t4.append(oracle_traces[3].flatten())
    v1t5.append(oracle_traces[4].flatten())
    v1t6.append(oracle_traces[5].flatten())
    v1t7.append(oracle_traces[6].flatten())
    v1t8.append(oracle_traces[7].flatten())
    v1t9.append(oracle_traces[8].flatten())
    v1t10.append(oracle_traces[9].flatten())





/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [ ]:
v1t1= np.array(v1t1)
v1t2= np.array(v1t2)
v1t3= np.array(v1t3)
v1t4= np.array(v1t4)
v1t5= np.array(v1t5)
v1t6= np.array(v1t6)
v1t7= np.array(v1t7)
v1t8= np.array(v1t8)
v1t9= np.array(v1t9)
v1t10= np.array(v1t10)
np.savetxt('area_v1_trial1_1_to_.csv', v1t1, delimiter= ',')
np.savetxt('area_v1_trial2_1_to_.csv', v1t2, delimiter= ',')
np.savetxt('area_v1_trial3_1_to_.csv', v1t3, delimiter= ',')
np.savetxt('area_v1_trial4_1_to_.csv', v1t4, delimiter= ',')
np.savetxt('area_v1_trial5_1_to_.csv', v1t5, delimiter= ',')
np.savetxt('area_v1_trial6_1_to_.csv', v1t6, delimiter= ',')
np.savetxt('area_v1_trial7_1_to_.csv', v1t7, delimiter= ',')
np.savetxt('area_v1_trial8_1_to_.csv', v1t8, delimiter= ',')
np.savetxt('area_v1_trial9_1_to_.csv', v1t9, delimiter= ',')
np.savetxt('area_v1_trial10_1_to_.csv', v1t10, delimiter= ',')


This loop follows the same procedure as the one described in the layers data gathering file. You can look there for the explanation.

## Area LM

In [6]:
df_area_lm = df_areas.loc[df_areas['brain_area'] == 'LM']
df_area_lm

,session,scan_idx,unit_id,brain_area
111,4,7,2711,LM
124,4,7,2763,LM
566,4,7,7716,LM
573,4,7,7748,LM
961,5,3,6766,LM
1291,5,6,1485,LM
1420,5,6,2390,LM
1432,5,6,2460,LM
1751,5,6,6752,LM
1854,5,6,7224,LM


In [ ]:
lmt1, lmt2, lmt3, lmt4, lmt5, lmt6, lmt7, lmt8, lmt9, lmt10 = ([] for i in range(10))
for row in range(len(df_area_lm)):
    unit_key = {'session': df_area_lm.iloc[row]['session'], 'scan_idx': df_area_lm.iloc[row]['scan_idx'], 'unit_id': df_area_lm.iloc[row]['unit_id']}
    oracle_traces, scores = utils.fetch_oracle_raster(unit_key)
    lmt1.append(oracle_traces[0].flatten())
    lmt2.append(oracle_traces[1].flatten())
    lmt3.append(oracle_traces[2].flatten())
    lmt4.append(oracle_traces[3].flatten())
    lmt5.append(oracle_traces[4].flatten())
    lmt6.append(oracle_traces[5].flatten())
    lmt7.append(oracle_traces[6].flatten())
    lmt8.append(oracle_traces[7].flatten())
    lmt9.append(oracle_traces[8].flatten())
    lmt10.append(oracle_traces[9].flatten())





/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [ ]:
lmt1= np.array(lmt1)
lmt2= np.array(lmt2)
lmt3= np.array(lmt3)
lmt4= np.array(lmt4)
lmt5= np.array(lmt5)
lmt6= np.array(lmt6)
lmt7= np.array(lmt7)
lmt8= np.array(lmt8)
lmt9= np.array(lmt9)
lmt10= np.array(lmt10)
np.savetxt('area_lm_trial1.csv', lmt1, delimiter= ',')
np.savetxt('area_lm_trial2.csv', lmt2, delimiter= ',')
np.savetxt('area_lm_trial3.csv', lmt3, delimiter= ',')
np.savetxt('area_lm_trial4.csv', lmt4, delimiter= ',')
np.savetxt('area_lm_trial5.csv', lmt5, delimiter= ',')
np.savetxt('area_lm_trial6.csv', lmt6, delimiter= ',')
np.savetxt('area_lm_trial7.csv', lmt7, delimiter= ',')
np.savetxt('area_lm_trial8.csv', lmt8, delimiter= ',')
np.savetxt('area_lm_trial9.csv', lmt9, delimiter= ',')
np.savetxt('area_lm_trial10.csv', lmt10, delimiter= ',')


## Area AL

In [6]:
df_area_al = df_areas.loc[df_areas['brain_area'] == 'AL']
df_area_al

,session,scan_idx,unit_id,brain_area
64,4,7,2573,AL
65,4,7,2574,AL
66,4,7,2585,AL
68,4,7,2594,AL
69,4,7,2605,AL
...,...,...,...,...
8090,8,5,8749,AL
8093,8,5,9762,AL
8094,8,5,9774,AL
8152,8,5,10241,AL


In [ ]:
alt1, alt2, alt3, alt4, alt5, alt6, alt7, alt8, alt9, alt10 = ([] for i in range(10))
for row in range(len(df_area_al)):
    unit_key = {'session': df_area_al.iloc[row]['session'], 'scan_idx': df_area_al.iloc[row]['scan_idx'], 'unit_id': df_area_al.iloc[row]['unit_id']}
    oracle_traces, scores = utils.fetch_oracle_raster(unit_key)
    alt1.append(oracle_traces[0].flatten())
    alt2.append(oracle_traces[1].flatten())
    alt3.append(oracle_traces[2].flatten())
    alt4.append(oracle_traces[3].flatten())
    alt5.append(oracle_traces[4].flatten())
    alt6.append(oracle_traces[5].flatten())
    alt7.append(oracle_traces[6].flatten())
    alt8.append(oracle_traces[7].flatten())
    alt9.append(oracle_traces[8].flatten())
    alt10.append(oracle_traces[9].flatten())





/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [ ]:
alt1= np.array(alt1)
alt2= np.array(alt2)
alt3= np.array(alt3)
alt4= np.array(alt4)
alt5= np.array(alt5)
alt6= np.array(alt6)
alt7= np.array(alt7)
alt8= np.array(alt8)
alt9= np.array(alt9)
alt10= np.array(alt10)
np.savetxt('area_al_trial1.csv', alt1, delimiter= ',')
np.savetxt('area_al_trial2.csv', alt2, delimiter= ',')
np.savetxt('area_al_trial3.csv', alt3, delimiter= ',')
np.savetxt('area_al_trial4.csv', alt4, delimiter= ',')
np.savetxt('area_al_trial5.csv', alt5, delimiter= ',')
np.savetxt('area_al_trial6.csv', alt6, delimiter= ',')
np.savetxt('area_al_trial7.csv', alt7, delimiter= ',')
np.savetxt('area_al_trial8.csv', alt8, delimiter= ',')
np.savetxt('area_al_trial9.csv', alt9, delimiter= ',')
np.savetxt('area_al_trial10.csv', alt10, delimiter= ',')


## Area RL

In [7]:
df_area_rl = df_areas.loc[df_areas['brain_area'] == 'RL']
df_area_rl

,session,scan_idx,unit_id,brain_area
0,4,7,648,RL
3,4,7,665,RL
29,4,7,921,RL
45,4,7,999,RL
53,4,7,1040,RL
...,...,...,...,...
8187,8,5,10464,RL
8188,8,5,10473,RL
8192,8,5,10550,RL
8193,8,5,10567,RL


In [ ]:
rlt1, rlt2, rlt3, rlt4, rlt5, rlt6, rlt7, rlt8, rlt9, rlt10 = ([] for i in range(10))
for row in range(len(df_area_rl)):
    unit_key = {'session': df_area_rl.iloc[row]['session'], 'scan_idx': df_area_rl.iloc[row]['scan_idx'], 'unit_id': df_area_rl.iloc[row]['unit_id']}
    oracle_traces, scores = utils.fetch_oracle_raster(unit_key)
    rlt1.append(oracle_traces[0].flatten())
    rlt2.append(oracle_traces[1].flatten())
    rlt3.append(oracle_traces[2].flatten())
    rlt4.append(oracle_traces[3].flatten())
    rlt5.append(oracle_traces[4].flatten())
    rlt6.append(oracle_traces[5].flatten())
    rlt7.append(oracle_traces[6].flatten())
    rlt8.append(oracle_traces[7].flatten())
    rlt9.append(oracle_traces[8].flatten())
    rlt10.append(oracle_traces[9].flatten())





/usr/local/lib/python3.9/site-packages/datajoint/connection.py:219: UserWarning: MySQL server has gone away. Reconnecting to the server.
  warnings.warn("MySQL server has gone away. Reconnecting to the server.")


In [ ]:
rlt1= np.array(rlt1)
rlt2= np.array(rlt2)
rlt3= np.array(rlt3)
rlt4= np.array(rlt4)
rlt5= np.array(rlt5)
rlt6= np.array(rlt6)
rlt7= np.array(rlt7)
rlt8= np.array(rlt8)
rlt9= np.array(rlt9)
rlt10= np.array(rlt10)
np.savetxt('area_rl_trial1_1_to_.csv', rlt1, delimiter= ',')
np.savetxt('area_rl_trial2_1_to_.csv', rlt2, delimiter= ',')
np.savetxt('area_rl_trial3_1_to_.csv', rlt3, delimiter= ',')
np.savetxt('area_rl_trial4_1_to_.csv', rlt4, delimiter= ',')
np.savetxt('area_rl_trial5_1_to_.csv', rlt5, delimiter= ',')
np.savetxt('area_rl_trial6_1_to_.csv', rlt6, delimiter= ',')
np.savetxt('area_rl_trial7_1_to_.csv', rlt7, delimiter= ',')
np.savetxt('area_rl_trial8_1_to_.csv', rlt8, delimiter= ',')
np.savetxt('area_rl_trial9_1_to_.csv', rlt9, delimiter= ',')
np.savetxt('area_rl_trial10_1_to_.csv', rlt10, delimiter= ',')
